# TP 1 : K-Nearest-Neighbours

In [ ]:
#imports
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from time import time
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
#load dataset
mnist = fetch_openml('mnist_784')

## Partie 1 : Exploration du jeu de données
### 1.1 Inspection de la structure du jeu de données

On analyse la nature et les dimensions des données à notre disposition. Le rôle et la valeur attendue est indiquée en commentaire pour chaque propriété

In [ ]:
#1.1
print(mnist) #représentation des paramètres du jeu de données, les images, leurs labels,
#et des informations supplémentaires de numpy (9 champs en tout)

print (mnist.data) #les images(chiffres) du dataset
print (mnist.target) #les labels(chiffre représenté par l'image) du dataset, 1 pour chaque image
print (len(mnist.data)) #le nombre d'images du dataset
print (mnist.data.shape) #la dimension des images du dataset (70000 * 784)
print (mnist.target.shape) #la dimension des labels du dataset (70000 * 1)
print (mnist.data[0]) #la première image du dataset
print (mnist.data[0][1]) #le deuxième pixel de la première image du dataset
print (mnist.data[:,1]) #le deuxième pixel de chaque image du dataset
print (mnist.data[:100]) #les 100 premières images du dataset

### 1.2 Affichage d'une image et du label associé
Plot d'une image : Puisque les images du dataset sont 'écrasées' en une dimension de 784 pixel, on les redimensionne d'abord au format ligne/colonne 28 * 28

In [ ]:
img_index = 0
images = mnist.data.reshape((-1, 28, 28))
plt.imshow(images[img_index],cmap=plt.cm.gray_r,interpolation="nearest")
print("Associated label :",mnist.target[img_index])

### 1.3 Extraction de données aléatoires
Enfin, on utilise les fonctions numpy pour extraire un échantillon aléatoire depuis le jeu de donénes initial, qui servira pour la suite du TP

In [ ]:
#choose sample size
dataset_length = len(mnist.data)
sample_size = 5000
sample_size = min(dataset_length,sample_size)

#extract sample from dataset
sample_indexes = np.random.randint(dataset_length, size= sample_size)
data, target = np.array([mnist.data[i] for i in sample_indexes]), np.array([mnist.target[i] for i in sample_indexes])

## Partie 2 : Méthode KNN
### 2.1 Premières classifications

#### Diviser la base de données à 80% pour l’apprentissage (training) et à 20% pour les tests

In [ ]:
train_size = 0.8
d_train, d_test, l_train, l_test = train_test_split(data,target,train_size = train_size)

#### Entrainer un classifieur k-nn avec k = 10 sur le jeu de données chargé.

In [ ]:
clf = KNeighborsClassifier(10)
clf.fit(d_train,l_train)

#### Afficher la classe de l’image 4 et sa classe prédite

In [ ]:
pred_index = 3
prediction = clf.predict(([data[pred_index]]))[0] #predict method awaits a list of data
expected = target[pred_index]
print(prediction, expected)

#### Afficher le score sur l’échantillon de test

In [ ]:
score = clf.score(d_test, l_test)
print(score)

#### Quel est le taux d'erreur sur vos données d'apprentissage ? Est-ce normal ?

In [ ]:
score = clf.score(d_train, l_train)
print(score)

On obtient un taux d'erreur non-nul (score < 100%)
C'était à attendre, puisque le modèle ne n'ajuste pas pour vérifier chaque échantillon du jeu de test, seulement un maximum d'entre eux

#### Faire varier le nombre de voisins (k) de 2 jusqu’à 15 et afficher le score. 
#### Quel est le k optimal ?

In [ ]:
best_score = 0
best_nb = 0 #k optimal
train_size = 0.8
d_train, d_test, l_train, l_test = train_test_split(data,target,train_size = train_size)
n_nb_set = range(2,16)
plot=([],[])

start = time() #TIMER START

for n_nb in n_nb_set:
    #print("Computing for",n_nb,train_size)
    clf = KNeighborsClassifier(n_nb)
    clf.fit(d_train	,l_train)
    score = clf.score(d_test, l_test)
    #print("Score :",score)
    if score>best_score:
        best_nb = n_nb
        best_score = score
    plot[0].append(n_nb)
    plot[1].append(score)
    
duration = time() - start #TIMER END

fig,ax = plt.subplots()
ax.set_xlabel("k neighbours")
ax.set_ylabel("score (%)")
ax.set_title("Score en fonction du nombre de voisins pris en compte")
ax.plot(plot[0],plot[1])
ax.plot([best_nb],[best_score], marker= 'x', color='r')

print("Optimal k :", best_nb, "( score :",best_score,')')
print("Executed in",duration,'s')

Le nombre de voisins(k) optimal semble être 5. Ce sera donc la valeur retenue pour les prochains tests où l'on devra fixer ce paramètre.

On notera cependant que les scores obtenus sont très proches, et semblent plus stable autour de 9 voisins.
On observe une dgradation globale pour des valeurs supérieures à 9 voisins.

#### Faites varier le pourcentage des échantillons (training et test) et affichez le score. 
#### Quel est le pourcentage remarquable ?

In [ ]:
best_score = 0 
best_size = 0 #training sample size
train_size_set = [i*0.05 for i in range(6,17)] #variation between 30-80 by 5% steps
n_nb = 5 #from previous estimations
plot=([],[])

start = time() #TIMER START
clf = KNeighborsClassifier(n_nb)
for train_size in train_size_set:
    d_train, d_test, l_train, l_test = train_test_split(data,target,train_size = train_size)
    clf.fit(d_train	,l_train)
    score = clf.score(d_test, l_test)
    if score>best_score:
        best_score = score
        best_size = train_size
    plot[0].append(100*train_size)
    plot[1].append(score)
    
duration = time() - start #TIMER END

fig,ax = plt.subplots()
ax.set_xlabel("training size (%)")
ax.set_ylabel("score (%)")
ax.set_title("Score en fonction de la proportion de l'échantillon d'entraînement/test pour k = 5")
ax.plot(plot[0],plot[1])

print("Optimal training proportion :",best_size, "( score :",best_score,')')
print("Executed in",duration,'s')

On observe une croissance nette du score jusqu'à 65% du dataset utilisé pour l'entraînement, puis une chute jusqu'à 70%
Le score croit ensuite de nouveau passé 70%, mais le pourcentage de données alors utilisées pour le test perd en signification : si l'on prenait 99% de données d'entraînement, le score serait probablement idéal mais le modèle serait moins pertinent une fois appliqué sur d'autres données.
Un bon compromis efficacité/pertinence semble donc être 65% d'entraînement.

#### Faites varier les types de distances (p). 
#### Quelle est la meilleure distance ?

In [ ]:
best_dist = 0
best_score = 0
dist_set = [1,2,3]
train_size = 0.65 #from previous tests
n_nb = 5 #from previous tests
d_train, d_test, l_train, l_test = train_test_split(data,target,train_size = train_size)
plot=([],[])

start = time() #TIMER START

for dist in dist_set:
    start_iter = time()
    clf = KNeighborsClassifier(n_nb, p = dist)
    clf.fit(d_train	,l_train)
    score = clf.score(d_test, l_test)
    duration_iter = time()-start_iter
    print("Execution for metric",dist,":",duration_iter,"s")
    if score>best_score:
        best_score = score
        best_dist = dist
    plot[0].append(dist)
    plot[1].append(score)
    
duration = time() - start #TIMER END

fig,ax = plt.subplots()
ax.set_xlabel("minkowski metric (p)")
ax.set_ylabel("score (%)")
ax.set_title("Score en fonction de la proportion de la distance utilisée (1 = mahnattan, 2 = euclid)")
ax.plot(plot[0],plot[1])
print("Optimal distance :",best_dist, "( score :",best_score,')')
print("Executed in",duration,'s')

La métrique de distance optimale semble donc être obtenue pour la distance de minkowski 4, et la tendance semble croissante avec la métrique utilisée.
En revanche, les temps d'exécution explose (x10) lorsque l'on sort des deux métriques classiques manhattan et euclidienne (1 et 2 respectivement). La métrique pertinente semble donc être la distance euclidienne, compromis en efficacité et rapidité.

#### Fixez n_job à 1 puis à -1 et calculez le temps de chacun.

In [ ]:
train_size = 0.65 #from previous tests
n_nb = 5 #from previous tests
dist = 2
d_train, d_test, l_train, l_test = train_test_split(data,target,train_size = train_size)


clfA = KNeighborsClassifier(n_nb, p = dist, n_jobs = 1)
clfB = KNeighborsClassifier(n_nb, p = dist, n_jobs = -1)

start = time()
clfA.fit(d_train, l_train)
score = clf.score(d_test, l_test)
duration = time() - start
print("Execution for n_jobs = 1:",duration,"s")

start = time()
clfB.fit(d_train, l_train)
score = clf.score(d_test, l_test)
duration = time() - start
print("Execution for n_jobs = -1:",duration,"s")

#### A votre avis, quels sont les avantages et les inconvénients des k-nn : 
#### optimalité ? temps de calcul ? passage à l'échelle ?

Les k-nn sont efficaces  /!\ A compléter /!\

En revanche, il est nécessaire de définir un jeu d'entraînement suffisamment petit et néanmoins représentatif. En effet, plus le jeu d'entraînement sera grand, plus le nombre de mesure de distance à effectuer pour une prédiction sera élevé.

De plus, la définition de la métrique de distance est cruciale et doit également être un compromis pertinence/temps de calcul car elle seraexécutée de nombreuse fois et représente la quasi totalité de la complexité de l'algorithme.

# FIN TP 1